In [1]:
%run ../../_setup.py

import numpy as np
from netCDF4 import Dataset
from gsw import IPV_vs_fNsquared_ratio, f, Nsquared

In [2]:
# load ctd and gsw data
input_file = os.path.join(datadir, 'processed', 'ss9802', 'netcdf', 'ss9802_ctd_gsw.nc')
ctd = Dataset(input_file, 'r')

In [3]:
coriolis = f(ctd['lat'][:])

SA = ctd['SA'][:]
CT = ctd['CT'][:]
p = ctd['p'][:]
lat = ctd['lat'][:, 1]

# print(SA.shape == (lat.shape + p.shape))


# SA, CT, p = np.broadcast_arrays(SA, CT, p)

# print(SA.shape == (lat.shape + p.shape))

# print(SA.shape, (lat.shape + p.shape))
# if SA.ndim == 2 and p.ndim == 1 and SA.shape == (lat.shape + p.shape) and :
#     print(np.broadcast_arrays(SA, p))

p = np.broadcast_arrays(SA, p)[1]
lat = np.broadcast_arrays(SA, lat[:,np.newaxis])[1]

# mask = SA.mask | CT.mask
# N2 = np.ma.masked_invalid(Nsquared(SA=SA.data, CT=CT.data, p=p, lat=lat)[0])
# N2 = Nsquared(SA=SA[0,], CT=CT[0,], p=p)
# N2


Nsquared(SA=SA[0,].data, CT=CT[0,].data, p=p)

/home/janjaapmeijer/miniconda3/envs/ocean36/lib/python3.6/site-packages/gsw/stability.py:93: RuntimeWarning: divide by zero encountered in true_divide
  N2 = ((g_local**2) / (specvol_mid * db_to_pa * dp))
/home/janjaapmeijer/miniconda3/envs/ocean36/lib/python3.6/site-packages/gsw/stability.py:94: RuntimeWarning: invalid value encountered in multiply
  N2 *= (beta_mid*dSA - alpha_mid*dCT)


(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 array([[   2.,    4.,    6., ..., 1522., 1524., 1526.],
        [   2.,    4.,    6., ..., 1522., 1524., 1526.],
        [   2.,    4.,    6., ..., 1522., 1524., 1526.],
        ...,
        [   2.,    4.,    6., ..., 1522., 1524., 1526.],
        [   2.,    4.,    6., ..., 1522., 1524., 1526.],
        [   2.,    4.,    6., ..., 1522., 1524., 1526.]]))

In [10]:
def Nsquared(SA, CT, p, lat=None, axis=0):
    """
    Calculate the square of the buoyancy frequency.

    Parameters
    ----------
    SA : array-like
        Absolute Salinity, g/kg
    CT : array-like
        Conservative Temperature (ITS-90), degrees C
    p : array-like
        Sea pressure (absolute pressure minus 10.1325 dbar), dbar
    lat : array-like, 1-D, optional
        Latitude, degrees.
    axis : int, optional
        The dimension along which pressure increases.

    Returns
    -------
    N2 : array
        Buoyancy frequency-squared at pressure midpoints, 1/s.
        The shape along the pressure axis dimension is one
        less than that of the inputs.
    p_mid : array
        Pressure at midpoints of p, dbar.
        The array shape matches N2.

    """
    if lat is not None:
        if np.any((lat < -90) | (lat > 90)):
            raise ValueError('lat is out of range')
        try:
            SA, CT, p, lat = np.broadcast_arrays(SA, CT, p, lat)
        except ValueError:
            if SA.shape == (lat.shape + p.shape):
                lat = lat[:, np.newaxis]
            else:
                p = p[:, np.newaxis]
            SA, CT, p = np.broadcast_arrays(SA, CT, p)
            lat = np.broadcast_arrays(SA, lat)[1]
        g = grav(lat, p)
    else:
        SA, CT, p = np.broadcast_arrays(SA, CT, p)
        g = 9.7963  # (Griffies, 2004)

    db_to_pa = 1e4
    shallow = axis_slicer(SA.ndim, slice(-1), axis)
    deep = axis_slicer(SA.ndim, slice(1, None), axis)
    if lat is not None:
        g_local = 0.5 * (g[shallow] + g[deep])
    else:
        g_local = g

    dSA = SA[deep] - SA[shallow]
    dCT = CT[deep] - CT[shallow]
    dp = p[deep] - p[shallow]
    SA_mid = 0.5 * (SA[shallow] + SA[deep])
    CT_mid = 0.5 * (CT[shallow] + CT[deep])
    p_mid = 0.5 * (p[shallow] + p[deep])

    specvol_mid, alpha_mid, beta_mid = specvol_alpha_beta(SA_mid,
                                                          CT_mid, p_mid)

    N2 = ((g_local**2) / (specvol_mid * db_to_pa * dp))
    N2 *= (beta_mid*dSA - alpha_mid*dCT)

    return N2, p_mid

In [11]:
Nsquared(SA=SA[0,].data, CT=CT[0,].data, p=p)

NameError: name 'axis_slicer' is not defined

In [5]:
from gsw

'/home/janjaapmeijer/miniconda3/envs/ocean3/lib/python3.5/site-packages/gsw-3.2.0+1.g478eb6e-py3.5-linux-x86_64.egg/gsw/__init__.py'